In [ ]:
import cv2
import face_recognition
import pickle

# Đường dẫn tới tệp chứa các encoding đã được lưu
ENCODINGS_PATH = "encoding.pickle"

# Đường dẫn đến video cần kiểm tra
VIDEO_PATH = "/Users/macbook/Desktop/Face_Verification/test_video/IMG_9755_converted.mp4"

# Load encodings
data = pickle.load(open(ENCODINGS_PATH, "rb"))

# Mở video
video_capture = cv2.VideoCapture(VIDEO_PATH)

# Kiểm tra xem video có mở thành công không
if not video_capture.isOpened():
    print("Error: Could not open video.")
    exit()

# Hàm cập nhật khung hình trong video
def update_frame(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Phát hiện các khuôn mặt trong khung hình
    boxes = face_recognition.face_locations(rgb_frame, model="hog")
    encodings = face_recognition.face_encodings(rgb_frame, boxes)

    for (box, encoding) in zip(boxes, encodings):
        matches = face_recognition.compare_faces(data["encodings"], encoding, tolerance=0.4)
        name = "Unknown"

        # Nếu có sự trùng khớp với encoding trong dataset
        if True in matches:
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}

            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1

            # Lấy tên của người có nhiều sự trùng khớp nhất
            name = max(counts, key=counts.get)

        # Vẽ bounding box và tên lên khung hình
        top, right, bottom, left = box
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    return frame

# Lặp qua từng khung hình video và cập nhật liên tục
while True:
    ret, frame = video_capture.read()
    if not ret:
        break  # Nếu không còn khung hình, thoát khỏi vòng lặp

    # Cập nhật khung hình với kết quả nhận diện khuôn mặt
    updated_frame = update_frame(frame)

    # Hiển thị khung hình trong cửa sổ OpenCV
    cv2.imshow("Video", updated_frame)

    # Dừng lại khi nhấn phím 'q' để thoát
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Giải phóng tài nguyên
video_capture.release()
cv2.destroyAllWindows()  # Đóng tất cả cửa sổ OpenCV


: 